In [1]:
#导入sqlite3数据库
import sqlite3 as sq3

#在sqlite3数据库上存取数据

##SQLite 是一个软件库，实现了自给自足的、无服务器的、零配置的、事务性的 SQL 数据库引擎。SQLite 是在世界上最广泛部署的 SQL 数据库引擎。可以直接在本地以一个文件的形式保存这个数据库。
##数据类型
##NULL	值是一个 NULL 值。
##INTEGER	值是一个带符号的整数，根据值的大小存储在 1、2、3、4、6 或 8 字节中。
##REAL	值是一个浮点值，存储为 8 字节的 IEEE 浮点数字。
##TEXT	值是一个文本字符串，使用数据库编码（UTF-8、UTF-16BE 或 UTF-16LE）存储。
##BLOB	值是一个 blob 数据，完全根据它的输入存储。

##创建数据库和表

In [6]:
path  = r'C:\Users\Administrator\Desktop\\'
#先在桌面创建一个数据库，直接连接没有就会创建一个
#connect 连接一个数据库
con =  sq3.connect(path+'data.db')
#创建一个表 并说明表内容的数据类型(列名 列类型,)
con.execute('CREATE TABLE numbs (Date date,No1 real,No2 real)')
#执行完都要提交
con.commit()

##写入数据

In [7]:
import datetime as dt
con.execute('INSERT INTO numbs VALUES(?,?,?)',(dt.datetime.now(),0.15,8.9))

In [10]:
#继续写入多条数据
import numpy as np
data = np.random.standard_normal((10000,2)).round(5) #取一万行2列的标准正态分布的随机数
for row in data:
    con.execute('INSERT INTO numbs VALUES(?,?,?)',(dt.datetime.now(),row[0],row[1]))
con.commit()

In [11]:
#executemany 可以同时写入多条语句

##查询语句

In [12]:
con.execute('SELECT * FROM numbs').fetchmany(10) #获取前十条 fetchone 获取一条

[('2018-10-22 11:21:12.648815', 0.15, 8.9),
 ('2018-10-22 11:26:28.671815', 0.29718, 0.77922),
 ('2018-10-22 11:26:28.671815', -0.57827, -1.47588),
 ('2018-10-22 11:26:28.671815', -0.50187, 0.93039),
 ('2018-10-22 11:26:28.671815', 0.03735, -0.77777),
 ('2018-10-22 11:26:28.672815', -0.87427, 0.76802),
 ('2018-10-22 11:26:28.672815', -1.12873, -0.63251),
 ('2018-10-22 11:26:28.672815', 1.15961, -0.00587),
 ('2018-10-22 11:26:28.672815', 0.93483, -1.439),
 ('2018-10-22 11:26:28.672815', -1.75245, 1.86312)]

In [13]:
pointer = con.execute('SELECT * FROM numbs')
for i in range(5):
    print(pointer.fetchone())

('2018-10-22 11:21:12.648815', 0.15, 8.9)
('2018-10-22 11:26:28.671815', 0.29718, 0.77922)
('2018-10-22 11:26:28.671815', -0.57827, -1.47588)
('2018-10-22 11:26:28.671815', -0.50187, 0.93039)
('2018-10-22 11:26:28.671815', 0.03735, -0.77777)


In [26]:
con.close()#最后要关闭下

#numpy存取数据

In [23]:
import numpy
dtimes = np.arange('2015-01-01 00:00:00','2018-01-01 00:00:00',dtype='datetime64[m]')#从15年到18年分钟级别的时间序列
num = np.random.standard_normal((len(dtimes),2)).round(5)
dty = np.dtype([('Date','datetime64[m]'),('No1','f'),('No2','f')]) #创建一个格式
data = np.zeros(len(dtimes),dtype=dty)
data['Date'] = dtimes
data['No1'] = num[:,0]
data['No2'] = num[:,1]

In [31]:
data,len(data)

(array([('2015-01-01T00:00',  0.04097, -0.54075),
        ('2015-01-01T00:01', -0.75385,  1.41079),
        ('2015-01-01T00:02',  2.00047,  1.26664), ...,
        ('2017-12-31T23:57',  1.47982,  0.37779),
        ('2017-12-31T23:58', -0.58949,  0.03246),
        ('2017-12-31T23:59',  0.10904, -0.19098)],
       dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')]), 1578240)

In [25]:
%time np.save(path+'num',data)#1578240行3列数据

Wall time: 44 ms


In [29]:
#只用了44毫秒 读取用load用了33ms
%time np.load(path+'num.npy')

Wall time: 30 ms


array([('2015-01-01T00:00',  0.04097, -0.54075),
       ('2015-01-01T00:01', -0.75385,  1.41079),
       ('2015-01-01T00:02',  2.00047,  1.26664), ...,
       ('2017-12-31T23:57',  1.47982,  0.37779),
       ('2017-12-31T23:58', -0.58949,  0.03246),
       ('2017-12-31T23:59',  0.10904, -0.19098)],
      dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')])

In [59]:
#pandas 和 sql3

In [38]:
import pandas as pd
import sqlite3 as sq3

In [39]:
data = np.random.standard_normal((1000000,5)).round(5)
file = path+'numbers'

In [45]:
con = sq3.connect(file+'.db')
con.execute('CREATE TABLE number (No1 real,No2 real,No3 real,No4 real,No5 real)')
%time con.executemany('INSERT INTO number VALUES(?,?,?,?,?)',data)
con.commit()#存用了7.15s

Wall time: 7.15 s


In [46]:
#读取用了1.54S
%time temp = con.execute('SELECT * FROM number').fetchall()

Wall time: 1.54 s


In [54]:
%time temp = con.execute('SELECT * FROM number WHERE No1>0 AND No2>0').fetchall() #506ms

Wall time: 506 ms


In [51]:
%time data = pd.read_sql('SELECT * FROM number',con) #在刚那个连接下 用pandas来读取数据 2s

Wall time: 2 s


In [52]:
data.head()

,No1,No2,No3,No4,No5
0,-1.79826,0.82435,-1.40551,-0.36927,0.08287
1,-0.51695,0.45340,-1.34435,-0.42546,0.15916
2,0.91593,2.21501,1.42353,1.15832,-1.12079
3,1.47658,1.45143,0.07586,2.19290,0.84683
4,0.55358,0.49137,0.30631,-1.28332,-0.33700


In [58]:
%time data[(data['No1']>0) & (data['No2']>0)].head() #37ms 比每次从数据库调取快很多 所以可以将数据先取出在做处理

Wall time: 33 ms


,No1,No2,No3,No4,No5
2,0.91593,2.21501,1.42353,1.15832,-1.12079
3,1.47658,1.45143,0.07586,2.19290,0.84683
4,0.55358,0.49137,0.30631,-1.28332,-0.33700
5,0.21947,1.24194,-0.93061,-0.59302,-1.61158
6,0.32374,1.20406,-0.04163,-1.12897,-0.27521


In [60]:
con.close()

#pandas 存取数据

In [61]:
%time data.to_csv(file+'.csv') #6.77s存csv文件

Wall time: 6.77 s


In [63]:
#读取数据
%time datas = pd.read_csv(file+'.csv') #1.03s
#excel就不测试了 太慢了 分别用 pd.read_excel 和pd.to_excel存取

Wall time: 1.03 s


#Pytables

In [66]:
import tables as tb
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [82]:
h5 = tb.open_file(path+'tab.h5','w') #以读形式打开 没有即为创建一个tab.h5
rows = 2000000
row_des = {
    'Date':tb.StringCol(30,pos=1),
    'No1':tb.IntCol(pos=2),
    'No2':tb.IntCol(pos=3),
    'No3':tb.FloatCol(pos=4),
    'No4':tb.FloatCol(pos=5)
}
filters = tb.Filters(complevel=0) #压缩程度为0
tab = h5.create_table('/','ints_floats',row_des,title='Integers',expectedrows=rows,filters=filters)

In [68]:
tab

/ints_floats (Table(0,)) 'Integers'
  description := {
  "Date": StringCol(itemsize=30, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)

In [71]:
ran_int = np.random.randint(0,10000,size=(rows,2))

In [84]:
ran_flo = np.random.standard_normal((rows,2)).round(5)

In [85]:
pointer = tab.row
#写入数据
for i in range(rows):
    pointer['Date']=dt.datetime.now()
    pointer['No1']=ran_int[i,0]
    pointer['No2']=ran_int[i,1]
    pointer['No3']=ran_flo[i,0]
    pointer['No4']=ran_flo[i,1]
    pointer.append()
tab.flush()  #刷新相当于commit提交

In [86]:
tab

/ints_floats (Table(2000000,)) 'Integers'
  description := {
  "Date": StringCol(itemsize=30, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)

In [90]:
#用numpy写入
dty = np.dtype([('Date','S26'),('No1','<i4'),('No2','<i4'),('No3','<f8'),('No4','<f8')])
sarry = np.zeros(len(ran_int),dtype=dty)
sarry['Date'] = dt.datetime.now()
sarry['No1']=ran_int[i,0]
sarry['No2']=ran_int[i,1]
sarry['No3']=ran_flo[i,0]
sarry['No4']=ran_flo[i,1]
h5.create_table('/','from_numpy',sarry,title='Integers',expectedrows=rows,filters=filters)

/from_numpy (Table(2000000,)) 'Integers'
  description := {
  "Date": StringCol(itemsize=26, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (2621,)

In [91]:
h5

File(filename=C:\Users\Administrator\Desktop\\tab.h5, title='', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/from_numpy (Table(2000000,)) 'Integers'
  description := {
  "Date": StringCol(itemsize=26, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (2621,)
/ints_floats (Table(2000000,)) 'Integers'
  description := {
  "Date": StringCol(itemsize=30, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)

In [92]:
tab[:5]

array([(b'2018-10-23 10:05:16.151815',  239, 6731,  1.63503, -0.07538),
       (b'2018-10-23 10:05:16.151815', 3853, 5744, -1.49574,  1.51015),
       (b'2018-10-23 10:05:16.151815', 5531, 7600,  1.04293,  0.51946),
       (b'2018-10-23 10:05:16.151815', 9869, 5498, -0.99279, -1.40255),
       (b'2018-10-23 10:05:16.151815', 4127, 4792, -0.53687,  1.09767)],
      dtype=[('Date', 'S30'), ('No1', '<i4'), ('No2', '<i4'), ('No3', '<f8'), ('No4', '<f8')])

In [93]:
tab[:5]['No3']

array([ 1.63503, -1.49574,  1.04293, -0.99279, -0.53687])

In [94]:
#计算
np.sum(tab[:]['No4'])

-700.4649299999994

In [95]:
h5.close()

# with open 打开文件

In [98]:
#用with open 后面不用close关闭
#readlines 是将所有内容读出每一行为一个列表
with open(path+'numbers.csv','r') as f:
    content = f.readlines()
    for line in content[:5]:
        print(line)
    print(content[:10])

,No1,No2,No3,No4,No5

0,-1.79826,0.82435,-1.40551,-0.36927,0.08287

1,-0.51695,0.4534,-1.34435,-0.42546,0.15916

2,0.91593,2.21501,1.42353,1.15832,-1.12079

3,1.47658,1.45143,0.07586,2.1929,0.84683

[',No1,No2,No3,No4,No5\n', '0,-1.79826,0.82435,-1.40551,-0.36927,0.08287\n', '1,-0.51695,0.4534,-1.34435,-0.42546,0.15916\n', '2,0.91593,2.21501,1.42353,1.15832,-1.12079\n', '3,1.47658,1.45143,0.07586,2.1929,0.84683\n', '4,0.55358,0.49137,0.30631,-1.28332,-0.337\n', '5,0.21947,1.24194,-0.93061,-0.59302,-1.61158\n', '6,0.32374,1.20406,-0.04163,-1.12897,-0.27521\n', '7,1.10115,0.74315,-0.92262,0.11751,-1.86326\n', '8,-1.39651,-0.70339,-0.67668,0.07697,1.18436\n']


In [99]:
with open(path+'numbers.csv','r') as f:
    content = f.readlines()
    for line in content[:5]:
        print(line)

,No1,No2,No3,No4,No5

0,-1.79826,0.82435,-1.40551,-0.36927,0.08287

1,-0.51695,0.4534,-1.34435,-0.42546,0.15916

2,0.91593,2.21501,1.42353,1.15832,-1.12079

3,1.47658,1.45143,0.07586,2.1929,0.84683

